In [1]:
from qcutils.credential import load_provider
from qcutils.benchmarks import QASMBenchmark
from qcutils.pulse import CircAmpTimeSeries, BasisPulse
from qcutils.pulse.amptimeseries.metric import dist, min_circ_dist_in_list

from qiskit import QuantumCircuit, transpile, schedule, Aer
from qiskit.quantum_info import Operator
from qiskit.quantum_info import state_fidelity
import qiskit.quantum_info as qi
import numpy as np


provider = load_provider()
backend = provider.get_backend("ibm_lagos")

bm = QASMBenchmark(r"E:\Research\QASMBench", "small", num_qubits_list=list(range(8)), remove_final_measurements=True, do_transpile=False)

In [2]:
def transform_sx(circ):
    from qiskit.circuit import CircuitInstruction, Instruction, ParameterExpression
    import random

    def transformed_sx(qubit, num_sx):
        if num_sx % 4 == 1:
            sx_1 = QuantumCircuit(circ.num_qubits)
            sx_1.sx(qubit)
            sx_1_circ_instructions = sx_1.data
            return sx_1_circ_instructions
        elif num_sx % 4 == 2:
            sx_2 = QuantumCircuit(circ.num_qubits)
            sx_2.rz(-np.pi/2, qubit)
            sx_2.sx(qubit)
            sx_2.rz(np.pi/2, qubit)
            sx_2.sx(qubit)
            sx_2.rz(-np.pi/2, qubit)
            sx_2_circ_instructions = sx_2.data
            return sx_2_circ_instructions
        elif num_sx % 4 == 3:
            sx_3 = QuantumCircuit(circ.num_qubits)
            sx_3.rz(-np.pi, qubit)
            sx_3.sx(qubit)
            sx_3.rz(np.pi/2, qubit)
            sx_3.sx(qubit)
            sx_3.sx(qubit)
            sx_3.rz(-np.pi/2, qubit)
            sx_3_circ_instructions = sx_3.data
            return sx_3_circ_instructions
        elif num_sx % 4 == 0:
            sx_4 = QuantumCircuit(circ.num_qubits)
            sx_4.rz(-np.pi, qubit)
            sx_4.sx(qubit)
            sx_4.sx(qubit)
            sx_4.rz(np.pi/2, qubit)
            sx_4.sx(qubit)
            sx_4.rz(-np.pi/2, qubit)
            sx_4.sx(qubit)
            sx_4.rz(-np.pi/2, qubit)
            sx_4_circ_instructions = sx_4.data
            return sx_4_circ_instructions

    circ_instructions = []
    for circ_instruction in circ.data:
        if circ_instruction.operation.name == "sx":
            qubit_idx = circ.find_bit(circ_instruction.qubits[0]).index
            # print(qubit_idx)
            # print(transformed_sx(qubit_idx, 2))
            num_sx = random.randint(1, 4)
            circ_instructions += transformed_sx(qubit_idx, num_sx)
        else:
            circ_instructions.append(circ_instruction)
    # return circ_instructions

    # transformed_circ = QuantumCircuit(circ.num_qubits, circ.num_clbits)
    return QuantumCircuit.from_instructions(circ_instructions, qubits=circ.qubits, clbits=circ.clbits)

In [3]:
simulator = Aer.get_backend('aer_simulator')

print("Fidelity")
count = 0
for circ_name, circ in bm.items():
    try:
        circ = transpile(circ, backend)
        dm1 = qi.Statevector.from_instruction(circ)
        circ = transpile(transform_sx(circ), backend, optimization_level=0)
        dm2 = qi.Statevector.from_instruction(circ)
        
        print(f"{circ_name}: {state_fidelity(dm1, dm2)}")
        count += 1
    except:
        print(f"{circ_name} raises exception")

print(f"Number of success circuits: {count}/{len(bm)}")

Fidelity
deutsch_n2: 1.0000000000000009
dnn_n2: 1.0000000000000422
grover_n2: 1.0000000000000022
ipea_n2 raises exception
iswap_n2: 1.0000000000000022
quantumwalks_n2: 1.0000000000000067
basis_change_n3: 1.0000000000000067
fredkin_n3: 1.0000000000000013
linearsolver_n3: 1.000000000000004
qaoa_n3: 1.0000000000000049
teleportation_n3: 1.0000000000000018
toffoli_n3: 1.0000000000000018
wstate_n3: 1.0000000000000013
adder_n4: 1.0
basis_trotter_n4: 1.0000000000003233
bell_n4: 1.0000000000000093
cat_state_n4: 1.0000000000000004
hs4_n4: 1.0000000000000004
inverseqft_n4 raises exception
qft_n4: 1.0000000000000022
qrng_n4: 1.0000000000000022
variational_n4: 1.0000000000000058
vqe_n4: 1.0000000000000093
vqe_uccsd_n4: 1.0000000000000213
error_correctiond3_n5: 1.000000000000004
lpn_n5: 1.0000000000000009
pea_n5: 1.000000000000004
qec_en_n5: 1.0000000000000036
qec_sm_n5 raises exception
shor_n5 raises exception
qaoa_n6: 1.0000000000000595
simon_n6: 1.0000000000000022
vqe_uccsd_n6: 1.0000000000001878